In [1]:
import pandas as pd
from path import Path
from collections import Counter

In [2]:
file_path = "ds2_cleaned.csv"
ds2 = pd.read_csv(file_path, index_col=0)
ds2.head()

,pha,eccentricity,semi_major_axis(au),q(au),inclination(deg),long_ascending_node(deg),argument_perihelion(deg),mean_anomaly(deg),aphelion_distance(au)
spkid,,,,,,,,,
2000433,0,0.2227,1.458,1.133,10.83,304.30,178.90,246.90,1.78
2000719,0,0.5470,2.638,1.195,11.58,183.86,156.23,278.20,4.08
2000887,0,0.5705,2.473,1.062,9.39,110.43,350.49,86.61,3.88
2001036,0,0.5331,2.666,1.245,26.68,215.52,132.43,140.65,4.09
2001221,0,0.4358,1.919,1.082,11.88,171.32,26.64,261.04,2.75


In [3]:
# Remove pha status target from features data
y = ds2.pha
X = ds2.drop(columns=['pha'])

In [4]:
Counter(y)

Counter({0: 25614, 1: 2213})

In [5]:
# Normal train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [6]:
# Use the SMOTEENN technique to perform combination sampling on the data
# Count the resampled classes
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({0: 15986, 1: 22131})

In [7]:
# Fit a Logistic regression model using random undersampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [12]:
# Calculating the confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)



# Calculating the accuracy score
acc_score = accuracy_score(y_test, y_pred)

In [14]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, y_pred))


Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,4011,2375
Actual 1,141,430


Accuracy Score : 0.638349863446888
Classification Report
              precision    recall  f1-score   support

           0       0.97      0.63      0.76      6386
           1       0.15      0.75      0.25       571

    accuracy                           0.64      6957
   macro avg       0.56      0.69      0.51      6957
weighted avg       0.90      0.64      0.72      6957



In [15]:
# Calculate the Balanced Accuracy Score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.6905787506931482

In [16]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.97      0.63      0.75      0.76      0.69      0.47      6386
          1       0.15      0.75      0.63      0.25      0.69      0.48       571

avg / total       0.90      0.64      0.74      0.72      0.69      0.47      6957

